# To do
* Fix N'Keal Harry
* Find people who dont show up in inner joins due to spelling

In [1]:
###################### Import Packages #############################################
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from ffb_functions import *

In [5]:
# college data pull 1: college stats: school, conf, games, stats, year
try:
    college_0 = pd.read_csv('college_stats.csv')
except:
    college_0 = college_assembly(2000, 2020)
    college_0.to_csv('college_stats.csv', index = False)
    
## to help the join
college_0['School'] = college_0['School'].str.replace('Pitt', 'Pittsburgh')
college_0['School'] = college_0['School'].str.replace('Ole Miss', 'Mississippi')
college_0['School'] = college_0['School'].str.replace('State', 'St.')
college_0['School'] = college_0['School'].str.replace('UAB', 'Ala-Birmingham')
college_0['School'] = college_0['School'].str.replace('Western Michigan', 'West. Michigan')
college_0['School'] = college_0['School'].str.replace('UCF', 'Central Florida')
college_0['School'] = college_0['School'].str.replace('Texas Christian', 'TCU')


# college data pull 1: draft pick: college, age, some stats
try:
    rk_0 = pd.read_csv('draft_pick.csv')
except:
    rk_0 = rookie_assembly(2000, 2020)
    rk_0.to_csv('draft_pick.csv', index = False)
    
rk_0['Name'] = rk_0['Name'].str.replace('DJ Chark', 'D.J. Chark')
rk_0.loc[rk_0.Name == 'Devin Funchess', 'FantPos'] = 'WR'
# college data pull 2: combine numbers: school, height, weight, combine stats, position, name
try:
    comb_0 = pd.adfread_csv('combine.csv')
except:
    comb_0 = combine_assembly(2000, 2020)
    comb_0.to_csv('combine.csv', index = False)
    
# merge rookie names, draft capital with combine statistics
draft_and_combine = rk_0.merge(comb_0, on = ['Name', 'FantPos', 'Year'], how = 'outer')
draft_and_combine = draft_and_combine.loc[draft_and_combine.Name != 'Mike Williams'].reset_index(drop = True)

## merge rookie names and draft capital with college statistics, keep inner joins for now
draft_combine_stats = draft_and_combine.merge(college_0.drop(columns = 'Year'), on = ['Name', 'School'], how = 'inner')
draft_combine_stats = draft_combine_stats.loc[draft_combine_stats.Name != 'Mike Williams'].reset_index(drop = True)

# rookie option makes the years line up properly
try:
    adp_frame_rk = pd.read_csv('adp_frame_rk.csv')
except:
    adp_frame_rk = new_assembly(2004, 2020, 'rookie') 
    adp_frame_rk.to_csv('adp_frame_rk.csv', index = False)

# merge to adp for fantasy relevant players
test = draft_combine_stats.merge(adp_frame_rk[['Name', 'Year', 'Overall', 'Tm']], on = ['Name', 'Year'], how = 'inner')

try:
    adp_frame_alt_rk = pd.read_csv('adp_frame_alt_rk.csv')
except:
    adp_frame_alt_rk = ADP_assembly(2010, 2020, 'rookie')         # sometimes this link is a problem. to fix it, open up the link in a browser and run again
    adp_frame_alt_rk.to_csv('adp_frame_alt_rk.csv', index = False)

# get std metric   
test_2 = test.merge(adp_frame_alt_rk, on = ['Name', 'Year'], how = 'left')


try:
    database = pd.read_csv('profootballfocus.csv')
except:
    database = data_assembly(2004, 2019)                            # can go back to at least 2000, limited by adp data
    database.to_csv('profootballfocus.csv', index = False)          # save original db

    
## change columns to floats for metric creation
cols_to_change = ['VBD', 'PosRank', 'OvRank', 'Rk', 'Age'
                  , 'PPR', 'GS', 'G'
                  , 'PaTD', 'Cmp', 'Int', 'RuTD', 'ReTD'
                  , 'PaYds', 'RuYds', 'ReYds', 'Rec'
                  , 'RuY/A', 'ReYds/R', 'PaAtt', 'RuAtt', 'Tgt'
                  , '2PM', '2PP', 'Fmb', 'FL', 'TD.3']
for col in cols_to_change:
    database[col] = database[col].astype(float)


# get ppr next year
test_3 = test_2.merge(database[['Name', 'Year', 'PPR']], on = ['Name', 'Year'], how = 'left')
test_3 = test_3.drop('Tm_x', axis = 1).rename(columns = {'Tm_y': 'Tm'})

# impute 0 for NaN points next year
test_3['PPR'] = test_3['PPR'].fillna(0)

# impute mean std for rookies
test_3['Std.Dev'] = test_3['Std.Dev'].fillna(test_3['Std.Dev'].mean())

# impute age 22 for missing
test_3['Age'] = test_3['Age'].fillna(22)

# impute 254 for undrafted
test_3['Pick'] = test_3['Pick'].fillna(254)

# now need to model
test_3.to_csv('rookie_database.csv', index = False)